# Neural Machine Translation

* We will build a Neural Machine Translation (NMT) model to translate human-readable dates ("25th of June, 2009") into machine-readable dates ("2009-06-25"). 
* We will do this using an attention model, one of the most sophisticated sequence-to-sequence models. 

**Note**: this notebook was influnced by the deep learning specialization

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from mt_utils import *
import keras
import keras.backend as K
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.models import load_model, Model
from keras.layers import Input, Dense, LSTM, Activation, Lambda
from keras.layers import Bidirectional, RepeatVector, Concatenate, Permute, Dot, multiply

In [2]:
# define number of examples
m = 10000

# Load the data (Or actually create the data :D)
dataset, human_vocab, machine_vocab, inv_machine_vocab = load_dataset(m)

100%|██████████| 10000/10000 [00:00<00:00, 21091.56it/s]


In [3]:
# Let's have a look on the data
dataset[:5]

[('9 may 1998', '1998-05-09'),
 ('10.11.19', '2019-11-10'),
 ('9/10/70', '1970-09-10'),
 ('saturday april 28 1990', '1990-04-28'),
 ('thursday january 26 1995', '1995-01-26')]

We've loaded:
- `dataset`: a list of tuples of (human readable date, machine readable date).
- `human_vocab`: a python dictionary mapping all characters used in the human readable dates to an integer-valued index.
- `machine_vocab`: a python dictionary mapping all characters used in machine readable dates to an integer-valued index. 
- `inv_machine_vocab`: the inverse dictionary of `machine_vocab`, mapping from indices back to characters. 

In [4]:
# We assume 30 is the maximum length of the human readable date.
Tx = 30

# We will set Ty=10, "YYYY-MM-DD" is 10 characters long.
Ty = 10

# Preprocess the data and map the raw text data into the index values
X, Y, Xoh, Yoh = preprocess_data(dataset, human_vocab, machine_vocab, Tx, Ty)

# Print some shapes
print("X.shape:", X.shape)
print("Y.shape:", Y.shape)
print("Xoh.shape:", Xoh.shape)
print("Yoh.shape:", Yoh.shape)

X.shape: (10000, 30)
Y.shape: (10000, 10)
Xoh.shape: (10000, 30, 37)
Yoh.shape: (10000, 10, 11)


We now have:
- `X`: a processed version of the human readable dates in the training set.
    - Each character in X is replaced by an index (integer) mapped to the character using `human_vocab`. 
    - Each date is padded to ensure a length of $T_x$ using a special character (< pad >). 
    - `X.shape = (m, Tx)` where m is the number of training examples in a batch.
- `Y`: a processed version of the machine readable dates in the training set.
    - Each character is replaced by the index (integer) it is mapped to in `machine_vocab`. 
    - `Y.shape = (m, Ty)`. 
- `Xoh`: one-hot version of `X`
    - Each index in `X` is converted to the one-hot representation (if the index is 2, the one-hot version has the index position 2 set to 1, and the remaining positions are 0.
    - `Xoh.shape = (m, Tx, len(human_vocab))`
- `Yoh`: one-hot version of `Y`
    - Each index in `Y` is converted to the one-hot representation. 
    - `Yoh.shape = (m, Ty, len(machine_vocab))`. 
    - `len(machine_vocab) = 11` since there are 10 numeric digits (0 to 9) and the `-` symbol.

In [5]:
# Defined shared layers as global variables
repeator = RepeatVector(Tx)
concatenator = Concatenate(axis=-1)
densor1 = Dense(10, activation = "tanh")
densor2 = Dense(1, activation = "relu")
# We are using a custom softmax(axis = 1) loaded in this notebook
activator = Activation(softmax, name='attention_weights')
dotor = Dot(axes = 1)

In [6]:
def one_step_attention(a, s_prev):
    """
    Performs one step of attention
    
    Inputs:
        a -- hidden state output of the Bi-LSTM, numpy-array of shape (m, Tx, 2*n_a)
        s_prev -- previous hidden state of the (post-attention) LSTM, numpy-array of shape (m, n_s)
    
    Returns:
        context -- context vector, input of the next (post-attention) LSTM cell
    """
    
    # Use repeator to repeat s_prev to be of shape (m, Tx, n_s)
    s_prev = repeator(s_prev)
    
    # Concatenate a and s_prev on the last axis
    concat = concatenator([a, s_prev])
    
    # Use densor1 to propagate concat through a small NN
    e = densor1(concat)
    
    # Use densor2 to propagate e through a small fully-connected neural network to compute the "energies"
    energies = densor2(e)
    
    # Use "activator" on "energies" to compute the attention weights "alphas"
    alphas = activator(energies)
    
    # Use dotor together with "alphas" and "a", to compute the context vector
    context = dotor([alphas, a])
    
    return context

In [7]:
# Again define shared layers as global variables
n_a = 32 # number of units for the pre-attention, bi-directional LSTM's hidden state 'a'
n_s = 64 # number of units for the post-attention LSTM's hidden state "s"

post_activation_LSTM_cell = LSTM(n_s, return_state = True)
output_layer = Dense(len(machine_vocab), activation=softmax)

In [8]:
def modelf(Tx, Ty, n_a, n_s, human_vocab_size, machine_vocab_size):
    """
    Inputs:
        Tx -- length of the input sequence
        Ty -- length of the output sequence
        n_a -- hidden state size of the Bi-LSTM
        n_s -- hidden state size of the post-attention LSTM
        human_vocab_size -- size of the python dictionary "human_vocab"
        machine_vocab_size -- size of the python dictionary "machine_vocab"

    Returns:
        model -- Keras model instance
    """
    
    # Define inputs for the model
    X = Input(shape=(Tx, human_vocab_size))
    s0 = Input(shape=(n_s,), name='s0')
    c0 = Input(shape=(n_s,), name='c0')
    
    # Define s0 (initial hidden state) and c0 (initial cell state)
    s = s0
    c = c0
    
    # Initialize empty list of outputs
    outputs = []
    
    # Define the pre-attention Bi-LSTM.
    a = Bidirectional(LSTM(n_a, return_sequences=True))(X)
    
    # Iterate for Ty steps
    for t in range(Ty):
        
        # Perform one step of the attention mechanism to get back the context vector at step t
        context = one_step_attention(a, s)
        
        # Apply the post-attention LSTM cell to the "context" vector.
        s, _, c = post_activation_LSTM_cell(context, initial_state=[s, c])
        
        # Apply Dense layer to the hidden state output of the post-attention LSTM
        out = output_layer(s)
        
        # Append out to outputs list
        outputs.append(out)
        
    # Create model instance taking three inputs and returning the list of outputs.
    model = Model(inputs=[X, s0, c0], outputs=outputs)
    
    return model

### Create and Train the model

In [9]:
model = modelf(Tx, Ty, n_a, n_s, len(human_vocab), len(machine_vocab))
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 30, 37)]     0                                            
__________________________________________________________________________________________________
s0 (InputLayer)                 [(None, 64)]         0                                            
__________________________________________________________________________________________________
bidirectional (Bidirectional)   (None, 30, 64)       17920       input_1[0][0]                    
__________________________________________________________________________________________________
repeat_vector (RepeatVector)    (None, 30, 64)       0           s0[0][0]                         
                                                                 lstm[0][0]                   

In [ ]:
# Define optimizer
opt = Adam(lr=0.005, decay=0.01)

# Compile the model
model.compile(loss = 'categorical_crossentropy', optimizer = opt , metrics = ['accuracy'])

In [11]:
s0 = np.zeros((m, n_s))
c0 = np.zeros((m, n_s))
outputs = list(Yoh.swapaxes(0,1))

In [12]:
model.fit([Xoh, s0, c0], outputs, epochs=25, batch_size=128)

Epoch 1/25
79/79 [==============================] - 20s 62ms/step - loss: 20.7285 - dense_2_loss: 1.9194 - dense_2_1_loss: 1.7789 - dense_2_2_loss: 2.2249 - dense_2_3_loss: 2.7320 - dense_2_4_loss: 1.4360 - dense_2_5_loss: 1.7421 - dense_2_6_loss: 2.7472 - dense_2_7_loss: 1.4584 - dense_2_8_loss: 1.9633 - dense_2_9_loss: 2.7265 - dense_2_accuracy: 0.1910 - dense_2_1_accuracy: 0.4331 - dense_2_2_accuracy: 0.1846 - dense_2_3_accuracy: 0.0843 - dense_2_4_accuracy: 0.7742 - dense_2_5_accuracy: 0.1022 - dense_2_6_accuracy: 0.0126 - dense_2_7_accuracy: 0.8095 - dense_2_8_accuracy: 0.1314 - dense_2_9_accuracy: 0.0688
Epoch 2/25
79/79 [==============================] - 5s 63ms/step - loss: 11.9721 - dense_2_loss: 0.3819 - dense_2_1_loss: 0.3161 - dense_2_2_loss: 1.2658 - dense_2_3_loss: 2.4564 - dense_2_4_loss: 0.2483 - dense_2_5_loss: 0.7699 - dense_2_6_loss: 2.5404 - dense_2_7_loss: 0.2797 - dense_2_8_loss: 1.4075 - dense_2_9_loss: 2.3062 - dense_2_accuracy: 0.9136 - dense_2_1_accuracy: 0.92

79/79 [==============================] - 5s 61ms/step - loss: 2.3888 - dense_2_loss: 0.0453 - dense_2_1_loss: 0.0349 - dense_2_2_loss: 0.2443 - dense_2_3_loss: 0.4205 - dense_2_4_loss: 0.0072 - dense_2_5_loss: 0.0791 - dense_2_6_loss: 0.5404 - dense_2_7_loss: 0.0082 - dense_2_8_loss: 0.4514 - dense_2_9_loss: 0.5575 - dense_2_accuracy: 0.9862 - dense_2_1_accuracy: 0.9890 - dense_2_2_accuracy: 0.9132 - dense_2_3_accuracy: 0.8646 - dense_2_4_accuracy: 0.9999 - dense_2_5_accuracy: 0.9783 - dense_2_6_accuracy: 0.8556 - dense_2_7_accuracy: 1.0000 - dense_2_8_accuracy: 0.8393 - dense_2_9_accuracy: 0.8216
Epoch 15/25
79/79 [==============================] - 5s 61ms/step - loss: 2.1491 - dense_2_loss: 0.0413 - dense_2_1_loss: 0.0309 - dense_2_2_loss: 0.2199 - dense_2_3_loss: 0.3753 - dense_2_4_loss: 0.0070 - dense_2_5_loss: 0.0740 - dense_2_6_loss: 0.4927 - dense_2_7_loss: 0.0073 - dense_2_8_loss: 0.4123 - dense_2_9_loss: 0.4885 - dense_2_accuracy: 0.9882 - dense_2_1_accuracy: 0.9903 - dense_2_

In [14]:
keras.models.save_model(model, filepath='my_model.h5')

### Test the model

In [15]:
EXAMPLES = ['3 May 1979', '5 April 09', '21th of August 2016', 'Tue 10 Jul 2007', 'Saturday May 9 2018', 'March 3 2001', 'March 3rd 2001', '1 March 2001']
s00 = np.zeros((1, n_s))
c00 = np.zeros((1, n_s))
for example in EXAMPLES:
    source = string_to_int(example, Tx, human_vocab)
    #print(source)
    source = np.array(list(map(lambda x: to_categorical(x, num_classes=len(human_vocab)), source))).swapaxes(0,1)
    source = np.swapaxes(source, 0, 1)
    source = np.expand_dims(source, axis=0)
    prediction = model.predict([source, s00, c00])
    prediction = np.argmax(prediction, axis = -1)
    output = [inv_machine_vocab[int(i)] for i in prediction]
    print("source:", example)
    print("output:", ''.join(output),"\n")

source: 3 May 1979
output: 1979-05-03 

source: 5 April 09
output: 2009-04-04 

source: 21th of August 2016
output: 2016-08-22 

source: Tue 10 Jul 2007
output: 2007-07-10 

source: Saturday May 9 2018
output: 2018-05-09 

source: March 3 2001
output: 2001-03-03 

source: March 3rd 2001
output: 2001-03-03 

source: 1 March 2001
output: 2001-03-01 



# DONE :D